### OUTPUT COLUMN NAMES

In [ ]:
### ResStock 2024 ComStock 2024
import pandas as pd
e_st = 'out.electricity.'
r_st = '.energy_consumption'

df_order = pd.DataFrame({
    'no': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25],
    'emm_intersect': ["TRE","FRCC","MISW","MISC","MISE","MISS","ISNE","NYCW","NYUP","PJME","PJMW","PJMC","PJMD","SRCA","SRSE","SRCE","SPPS","SPPC","SPPN","SRSG","CANO","CASO","NWPP","RMRG","BASN"]
})

def get_scout_geo():
    file = 'scout_geography.csv'
    df = pd.read_csv(file, dtype=str)

    df['fips_code'] = df['fips_code'].str.zfill(5)
    df['gisjoin'] = df['fips_code'].apply(
        lambda x: 'G' + str(x)[:2] + '0' + str(x)[2:] + '0')
    df.to_csv(f'new_{file}', index=False)
    return df

scoutgeo_df = get_scout_geo()

indir = 'input'
outdir = 'output'
building_types = ['commercial','residential']
substrings = [".savings", "out.electricity.peak_", "out.electricity.total."]

weathers = ['amy2018']
for weath in weathers:
    df_comstock = pd.read_parquet(f'./{indir}/2024_comstock/{weath}/baseline_metadata_and_annual_results.parquet', engine='pyarrow')
    df_resstock = pd.read_parquet(f'./{indir}/2024_resstock/{weath}/baseline_metadata_and_annual_results.parquet', engine='pyarrow')


output_file = "DICT_ComStock2024.txt"

# Open the file in write mode and save the output
with open(output_file, 'w') as f:
    f.write(f'{weath} comstock\n')
    for col in df_comstock.columns:
        # if ("out.electricity." in col) and (all(sub not in col for sub in substrings)):
        if (all(sub not in col for sub in substrings)):
            f.write(f'{col}\n')

output_file = "DICT_ResStock2024.txt"

# Open the file in write mode and save the output
with open(output_file, 'w') as f:
    f.write(f'{weath} resstock\n')
    for col in df_resstock.columns:
        # if ("out.electricity." in col) and (all(sub not in col for sub in substrings)):
        if (all(sub not in col for sub in substrings)):
            f.write(f'{col}\n')


# print(f'{weath} comstock')
# for col in df_comstock.columns:
#     # if ("out.electricity." in col) and (all(sub not in col for sub in substrings)):
#     if (all(sub not in col for sub in substrings)):
#         print(col)
        
# print(f'{weath} resstock')
# for col in df_resstock.columns:
#     if ("out.electricity." in col) and (all(sub not in col for sub in substrings)):
#         print(col)

### COMBINE HVAC TECH AND OTHER END USE

In [ ]:
import pandas as pd
import os

tech_dir = "./output/2024_technology/"
end_use_dir = "./output/2024_end_use/"
filenames = [
    "Com_Cdiv_EMM_amy2018",
    "Com_Cdiv_State_amy2018",
    "Res_Cdiv_EMM_amy2018",
    "Res_Cdiv_State_amy2018",
    "Com_Cdiv_EMM_amy2018_Stock",
    "Com_Cdiv_State_amy2018_Stock",
    "Res_Cdiv_EMM_amy2018_Stock",
    "Res_Cdiv_State_amy2018_Stock"
    ]

for filename in filenames:
    # name, ext = os.path.splitext(filename)

    if "Stock" in filename:
        filename_eu = f"{filename.replace('Stock','')}electricity_Stock.csv"
        filename_tech = f"{filename.replace('Stock','')}electricity_Stock_Tech.csv"
    else:
        filename_eu = f"{filename}_electricity.csv"
        filename_tech = f"{filename}_electricity_Tech.csv"
    
    file_tech = os.path.join(tech_dir, filename_tech)
    file_enduse = os.path.join(end_use_dir, filename_eu)
    
    df_tech = pd.read_csv(file_tech)
    df_end_use = pd.read_csv(file_enduse)

    print(f"{filename_tech} {file_enduse}")
    
    # Filter out rows where "End use" is "Heating" or "Cooling" in df_all
    # df_end_use_filtered = df_end_use[~df_end_use['End use'].isin(['heating', 'cooling'])]
    df_end_use_filtered = df_end_use
    # Add the required columns to df_all_filtered with fixed values
    df_end_use_filtered['Technology'] = "all"
    
    # Append df_all_filtered to df_tech
    df_combined = pd.concat([df_tech, df_end_use_filtered], ignore_index=True)
    
    # Save the combined dataframe to a new CSV file (optional)
    output_file = f"./output/{filename_tech}"
    df_combined.to_csv(output_file, index=False)

print("COMPLETE")

### FILL NA with 0s

In [ ]:
import pandas as pd
import os

folder_paths = ["./output/2024_end_use","./output/2024_technology"] 

for folder_path in folder_paths:
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
    
            if df.isnull().values.any():
                df_filled = df.fillna(0)
                output_filename = filename.replace(".csv", ".csv")
                output_path = os.path.join(folder_path, output_filename)
                df_filled.to_csv(output_path, index=False)
                print(f"{filename}: Blank cells filled. Saved as {output_filename}")
            else:
                print(f"{filename}: No blank cell")


### DICTIONARY

In [ ]:
import pandas as pd

enduse_map = {
    "commercial": {
        "electricity": {
            "heating": [
                "out.electricity.heating.energy_consumption",
                "out.electricity.heat_recovery.energy_consumption"],
            "cooling": [
                "out.electricity.heat_rejection.energy_consumption",
                "out.electricity.cooling.energy_consumption",
                "out.district_cooling.cooling.energy_consumption"],
            "water heating": ["out.electricity.water_systems.energy_consumption"],
            "fans and pumps": [
                "out.electricity.fans.energy_consumption",
                "out.electricity.pumps.energy_consumption"],
            "lighting": [
                "out.electricity.interior_lighting.energy_consumption",
                "out.electricity.exterior_lighting.energy_consumption"],
            "refrigeration": ["out.electricity.refrigeration.energy_consumption"],
            "misc": ["out.electricity.interior_equipment.energy_consumption"]
        },
        "natural gas":{
            "heating": ["out.natural_gas.heating.energy_consumption",
                        "out.district_heating.heating.energy_consumption"],
            "cooling": ["out.natural_gas.heating.energy_consumption"],
            "water heating": ["out.natural_gas.water_systems.energy_consumption",
                              "out.district_heating.water_systems.energy_consumption"],
            "misc": ["out.natural_gas.interior_equipment.energy_consumption"]
        },
        "distillate": {
            "heating": ["out.other_fuel.heating.energy_consumption"],
            "cooling": ["out.other_fuel.cooling.energy_consumption"],
            "water heating": ["out.other_fuel.water_systems.energy_consumption"],
            "misc": ["out.natural_gas.interior_equipment.energy_consumption"] 
        },
        "other fuel": {
            "misc": ["out.natural_gas.interior_equipment.energy_consumption"] 
        }
    },
    "residential": {
        "electricity": {
            "heating": [
                "out.electricity.heating.energy_consumption.kwh",
                "out.electricity.heating_hp_bkup.energy_consumption.kwh"],
            "cooling": ["out.electricity.cooling.energy_consumption.kwh"],
            "water heating": ["out.electricity.hot_water.energy_consumption.kwh"],
            "cooking": ["out.electricity.range_oven.energy_consumption.kwh"],
            "drying": ["out.electricity.clothes_dryer.energy_consumption.kwh"],
            "clothes washing": ["out.electricity.clothes_washer.energy_consumption.kwh"],
            "dishwasher": ["out.electricity.dishwasher.energy_consumption.kwh"],
            "lighting": [
                "out.electricity.lighting_exterior.energy_consumption.kwh",
                "out.electricity.lighting_interior.energy_consumption.kwh",
                "out.electricity.lighting_garage.energy_consumption.kwh"],
            "refrigeration": [
                "out.electricity.freezer.energy_consumption.kwh",
                "out.electricity.refrigerator.energy_consumption.kwh"],
            "ceiling fan": ["out.electricity.ceiling_fan.energy_consumption.kwh"],
            "misc": ["out.electricity.plug_loads.energy_consumption.kwh"],
            "pool heaters": ["out.electricity.pool_heater.energy_consumption.kwh"],
            "pool pumps": ["out.electricity.pool_pump.energy_consumption.kwh"],
            "portable electric spas": [
                "out.electricity.permanent_spa_heat.energy_consumption.kwh",
                "out.electricity.permanent_spa_pump.energy_consumption.kwh"],
            "fans and pumps": [
                "out.electricity.mech_vent.energy_consumption.kwh",
                "out.electricity.cooling_fans_pumps.energy_consumption.kwh",
                "out.electricity.heating_fans_pumps.energy_consumption.kwh",
                "out.electricity.heating_hp_bkup_fa.energy_consumption.kwh",
                "out.electricity.well_pump.energy_consumption.kwh"],
        },
        "distillate": {
            "heating": [
                "out.fuel_oil.heating.energy_consumption.kwh",
                "out.fuel_oil.heating_hp_bkup.energy_consumption.kwh"],
            "water heating": ["out.fuel_oil.hot_water.energy_consumption.kwh"],
            "misc": ["out.natural_gas.pool_heater.energy_consumption.kwh"]
        },
        "other fuel": {
            "heating": [
                "out.propane.heating.energy_consumption.kwh",
                "out.propane.heating_hp_bkup.energy_consumption.kwh"],
            "water heating": ["out.propane.hot_water.energy_consumption.kwh"],
            "cooking": ["out.propane.range_oven.energy_consumption.kwh"],
            "misc": ["out.natural_gas.pool_heater.energy_consumption.kwh"],
            "drying": ["out.propane.clothes_dryer.energy_consumption.kwh"]
        },
        "natural gas": {
            "heating": [
                "out.natural_gas.heating.energy_consumption.kwh",
                "out.natural_gas.heating_hp_bkup.energy_consumption.kwh"],
            "cooling": [
                "out.natural_gas.heating.energy_consumption.kwh",
                "out.natural_gas.heating_hp_bkup.energy_consumption.kwh"],
            "water heating": ["out.natural_gas.hot_water.energy_consumption.kwh"],
            "cooking": [
                "out.natural_gas.grill.energy_consumption.kwh",
                "out.natural_gas.range_oven.energy_consumption.kwh"],
            "drying": ["out.natural_gas.clothes_dryer.energy_consumption.kwh"],
            "misc": ["out.natural_gas.pool_heater.energy_consumption.kwh"]
            # "pool heaters": ["out.natural_gas.pool_heater.energy_consumption.kwh"],
            # "portable electric spas": ["out.natural_gas.permanent_spa_heat.energy_consumption.kwh"]
        }
    }
}

df_order = pd.DataFrame({
    'no': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25],
    'emm_intersect': ["TRE","FRCC","MISW","MISC","MISE","MISS","ISNE","NYCW","NYUP","PJME","PJMW","PJMC","PJMD","SRCA","SRSE","SRCE","SPPS","SPPC","SPPN","SRSG","CANO","CASO","NWPP","RMRG","BASN"]
})

stock_map = {
    "commercial": {
        "stock": ["out.params.smallest_space_floor_area..m2"]
    },
    "residential": {
        "stock": ["in.units_represented"]
    }
}

In [ ]:
fuel_enduse_map = {
    "commercial": {
        "electricity": {
            "cooling": [
                "out.electricity.cooling.energy_consumption",
                "out.electricity.heat_rejection.energy_consumption"],
            "heating": [
                "out.electricity.heating.energy_consumption",
                "out.electricity.heat_recovery.energy_consumption"],
            # "fans and pumps": [
            #     "out.electricity.fans.energy_consumption",
            #     "out.electricity.pumps.energy_consumption"],
            # "water heating": ["out.electricity.water_systems.energy_consumption"],
            # "lighting": [
            #     "out.electricity.interior_lighting.energy_consumption",
            #     "out.electricity.exterior_lighting.energy_consumption"],
            # "refrigeration": ["out.electricity.refrigeration.energy_consumption"],
            # "plug loads": ["out.electricity.interior_equipment.energy_consumption"]
        },
        # "other fuel": {
        #     "cooling": [
        #         "out.other_fuel.cooling.energy_consumption"
        #     ],
        #     "heating": [
        #         "out.other_fuel.heating.energy_consumption"
        #     ],
        #     "water heating": [
        #         "out.other_fuel.water_systems.energy_consumption"
        #     ]
        # },
        # "natural_gas": {
        #     "plug loads": [
        #         "out.natural_gas.interior_equipment.energy_consumption"
        #     ],
        #     "heating": [
        #         "out.natural_gas.heating.energy_consumption"
        #     ],
        #     "water heating": [
        #         "out.natural_gas.water_systems.energy_consumption"
        #     ]
        # }
    },
    "residential": {
        "electricity": {
            "cooling": ["out.electricity.cooling.energy_consumption.kwh"],
            "heating": [
                "out.electricity.heating.energy_consumption.kwh",
                "out.electricity.heating_hp_bkup.energy_consumption.kwh"],
            # "water heating": ["out.electricity.hot_water.energy_consumption.kwh"],
            # "cooking": ["out.electricity.range_oven.energy_consumption.kwh"],
            # "drying": ["out.electricity.clothes_dryer.energy_consumption.kwh"],
            # "clothes washing": ["out.electricity.clothes_washer.energy_consumption.kwh"],
            # "dishwasher": ["out.electricity.dishwasher.energy_consumption.kwh"],
            # "lighting": [
            #     "out.electricity.lighting_exterior.energy_consumption.kwh",
            #     "out.electricity.lighting_interior.energy_consumption.kwh",
            #     "out.electricity.lighting_garage.energy_consumption.kwh"],
            # "refrigeration": [
            #     "out.electricity.freezer.energy_consumption.kwh",
            #     "out.electricity.refrigerator.energy_consumption.kwh"],
            # "ceiling fan": ["out.electricity.ceiling_fan.energy_consumption.kwh"],
            # "fans and pumps": [
            #     "out.electricity.mech_vent.energy_consumption.kwh",
            #     "out.electricity.cooling_fans_pumps.energy_consumption.kwh",
            #     "out.electricity.heating_fans_pumps.energy_consumption.kwh",
            #     "out.electricity.heating_hp_bkup_fa.energy_consumption.kwh",
            #     "out.electricity.well_pump.energy_consumption.kwh"],
            # "plug loads": ["out.electricity.plug_loads.energy_consumption.kwh"],
            # "pool heaters": ["out.electricity.pool_heater.energy_consumption.kwh"],
            # "pool pumps": ["out.electricity.pool_pump.energy_consumption.kwh"],
            # "portable electric spas": [
            #     "out.electricity.permanent_spa_heat.energy_consumption.kwh",
            #     "out.electricity.permanent_spa_pump.energy_consumption.kwh"]
        },
        # "oil": {
        #     "heating": [
        #         "out.fuel_oil.heating.energy_consumption.kwh",
        #         "out.fuel_oil.heating_hp_bkup.energy_consumption.kwh",
        #     ],
        #     "water heating": [
        #         "out.fuel_oil.hot_water.energy_consumption.kwh"
        #     ]
        # },
        # "natural_gas": {
        #     "drying": [
        #         "out.natural_gas.clothes_dryer.energy_consumption.kwh"
        #     ],
        #     "cooking": [
        #         "out.natural_gas.grill.energy_consumption.kwh",
        #         "out.natural_gas.range_oven.energy_consumption.kwh",
        #         "out.natural_gas.range_oven.energy_consumption.kwh"
        #     ],
        #     "heating": [
        #         "out.natural_gas.heating.energy_consumption.kwh",
        #         "out.natural_gas.heating_hp_bkup.energy_consumption.kwh"
        #     ],
        #     "water heating": [
        #         "out.natural_gas.hot_water.energy_consumption.kwh"
        #     ],
        #     "lighting": [
        #         "out.natural_gas.lighting.energy_consumption.kwh",
        #     ],
        #     "pool heaters": [
        #         "out.natural_gas.pool_heater.energy_consumption.kwh"
        #     ],
        #     "portable electric spas": [
        #         "out.natural_gas.permanent_spa_heat.energy_consumption.kwh"
        #     ]
        # },
        # "propane": {
        #     "heating": [
        #         "out.propane.heating.energy_consumption.kwh",
        #         "out.propane.heating_hp_bkup.energy_consumption.kwh",
        #     ],
        #     "water heating" : [
        #         "out.propane.hot_water.energy_consumption.kwh"
        #     ],
        #     "drying": [
        #         "out.propane.clothes_dryer.energy_consumption.kwh"
        #     ],
        #     "cooking": [
        #         "out.propane.range_oven.energy_consumption.kwh"
        #     ]
        # }
    }}

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

def remove_space(string):
    return string.replace(" ", "")

def get_scout_geo():
    file = 'scout_geography.csv'
    df = pd.read_csv(file, dtype=str)
    
    df['fips_code'] = df['fips_code'].str.zfill(5)
    df['gisjoin'] = df['fips_code'].apply(
        lambda x: 'G' + str(x)[:2] + '0' + str(x)[2:] + '0')
    # df.loc[df['state'] == 'AK', 'gisjoin'] = 'G0'
    # df.loc[df['state'] == 'HI', 'gisjoin'] = 'G1'
    
    df.loc[df['state'] == 'AK', 'emm2020_county'] = 'AK_HI'
    df.loc[df['state'] == 'HI', 'emm2020_county'] = 'AK_HI'
    df.to_csv(f'new_{file}', index=False)
    return df


def transpose(df):
    return df.transpose()

def combine_keys(_dict):
    combined_dict = {}

    for fuel_type, categories in _dict.items():
        for category, values in categories.items():
            combined_key = f"{fuel_type}_{category.replace(' ', '_')}"
            combined_dict[combined_key] = values
    
    return combined_dict
def contains_utf8(text):
    try:
        text.encode('ascii')  # Try encoding to ASCII (UTF-8 characters will fail)
        return False
    except UnicodeEncodeError:
        return True

### CDIV/EMM - ENERGY (TECH)

In [ ]:
def apply_geographies(df, dfdict, geos):
    df['county'] = df['county'].astype(str)
    df.loc[df['state'] == 'AK', 'county'] = 'G0'
    df.loc[df['state'] == 'HI', 'county'] = 'G1'        
    for geo in geos:
        geocol = 'emm2020_county' if geo == 'emm' else geo
        d = dfdict.set_index('gisjoin').T.to_dict('index')[geocol]
        df[geo] = df['county'].map(d)
    df = df.drop('county', axis=1)
    return df

def output(df):
    merged_matrix = df.merge(df_order, left_on='emm', right_on='emm_intersect', how='left')
    sorted_matrix = merged_matrix.sort_values(by='no')
    sorted_matrix = sorted_matrix.drop(sorted_matrix.columns[-2:], axis=1)
    sorted_matrix.loc['total'] = sorted_matrix.sum()
    sorted_matrix = transpose(sorted_matrix)  ### to transpose CDIV on the row and EMM on the columns

    return(sorted_matrix)


def replace_col_vals(df, tech):
    df = df.copy()
    df.drop(columns=['Technology'], inplace=True)
    df.insert(0, 'Technology', tech)
    return df
    
def convert_matrix(sector,filedir, filename, weathers, mymap):

    if sector == "commercial":
        county_col = "in.nhgis_county_gisjoin"
        sec = "Com"
    else:
        county_col = "in.county"
        sec = "Res"
        
    combined_map = combine_keys(mymap[sector])
    mykeys = list(combined_map)
    
    for weath in weathers:
        # weath = 'tmy3'
        df_all = pd.read_parquet(f"{filedir}{weath}/{filename}", engine='pyarrow')
        df_all.rename(columns={county_col: 'county'}, inplace=True)
        df_all.rename(columns={'in.state': 'state'}, inplace=True)
        
        df_all.reset_index(inplace=True)
        df_all = apply_geographies(df_all, scoutgeo_df, geos)
        df_all = df_all.dropna(subset=geos)

        all_eu = pd.DataFrame()
        for eu in mykeys:
            df = df_all.copy()
            df[eu] = df[combined_map[eu]].sum(axis=1)
            df = df[df[eu]>0]
            mapfile = f"map_{sector[:3]}_{eu}.csv"
            map_df = pd.read_csv(f"./{fdir}/mapping/{mapfile}")

            df['scout_tech'] = None
            for _, map_row in map_df.iterrows():
                condition = pd.Series(True, index=df.index)
                for col in map_df.columns:
                    if col != 'scout_tech' and col in df.columns:
                        condition &= (df[col] == map_row[col])
                df.loc[condition, 'scout_tech'] = map_row['scout_tech']
            matched_rows = df[df['scout_tech'].notna()]
            columns_to_keep = geos + ['scout_tech'] + [eu]
            df = df[columns_to_keep]
            tech_list = df['scout_tech'].unique().tolist()
            # ######## DEBUG ##################
            # print(f"END USE {eu}")
            # print(df.shape[0])
            # with pd.option_context('display.max_columns', None):
            #     for col in df.columns:
            #         print(col)
            # ######## DEBUG ##################
            
            df = df.groupby(['emm', 'cdiv', 'scout_tech']).sum().reset_index()

            all_tech = pd.DataFrame()
            for tech in tech_list:

                convert_pd = pd.DataFrame()
                tdf = df[(df['scout_tech'] == tech)]
                tdf = tdf.drop(columns=['scout_tech'])
                conversion_matrix = tdf.pivot(index='emm', columns='cdiv', values=eu)
                convert_matrix = output(conversion_matrix.reset_index())
                convert_pd = convert_matrix if convert_pd.empty else pd.concat([convert_pd, convert_matrix], ignore_index=False)
                normalized_matrix = conversion_matrix.div(conversion_matrix.sum(axis=0), axis=1).reset_index()
                normalized_matrix = output(normalized_matrix)
                normalized_matrix = normalized_matrix.fillna(0)
                normalized_matrix.columns = normalized_matrix.iloc[0]
                normalized_matrix.rename(columns={normalized_matrix.columns[-1]: 'Total'}, inplace=True)
                normalized_matrix = normalized_matrix.iloc[1:]
                normalized_matrix.insert(0, 'CDIV', normalized_matrix.index)
                normalized_matrix.insert(0, 'End use',  eu.split('_')[1])
                normalized_matrix.insert(0, 'Technology', tech)
                if (normalized_matrix['Total'] != 0).all():
                    normalized_matrix.drop(columns=['Total'], inplace=True)
                    all_tech = normalized_matrix if all_tech.empty else pd.concat([all_tech, normalized_matrix], ignore_index=False)
                    if tech == "res_type_central_AC":
                        norm2 = replace_col_vals(normalized_matrix, "wall-window_room_AC")
                        all_tech = pd.concat([all_tech, norm2], ignore_index=False)
            all_eu = all_tech if all_eu.empty else pd.concat([all_eu, all_tech], ignore_index=False)
        all_eu.to_csv(f"./{outdir}/{sec}_Cdiv_EMM_{weath}__electricity_Tech.csv", index=False)


scoutgeo_df = get_scout_geo()

fdir = 'input'
outdir = 'output/2024_technology'
geos = ['emm', 'cdiv']

print("COMMERCIAL")
convert_matrix(
    sector = "commercial",
    filedir = f"./{fdir}/2024_comstock/",
    filename = "baseline_metadata_and_annual_results.parquet",
    weathers = ['amy2018'],
    mymap = fuel_enduse_map)
print("RESIDENTIAL")
convert_matrix(
    sector = "residential",
    filedir = f"./{fdir}/2024_resstock/",
    filename = "baseline_metadata_and_annual_results.parquet",
    weathers = ['amy2018'],
    mymap = fuel_enduse_map)

print("COMPLETE")


### CDIV/EMM - STOCK (TECH)

In [ ]:
def apply_geographies(df, dfdict, geos):
    df['county'] = df['county'].astype(str)
    df.loc[df['state'] == 'AK', 'county'] = 'G0'
    df.loc[df['state'] == 'HI', 'county'] = 'G1'        
    for geo in geos:
        geocol = 'emm2020_county' if geo == 'emm' else geo
        d = dfdict.set_index('gisjoin').T.to_dict('index')[geocol]
        df[geo] = df['county'].map(d)
    df = df.drop('county', axis=1)
    return df

def output(df):
    merged_matrix = df.merge(df_order, left_on='emm', right_on='emm_intersect', how='left')
    sorted_matrix = merged_matrix.sort_values(by='no')
    sorted_matrix = sorted_matrix.drop(sorted_matrix.columns[-2:], axis=1)
    sorted_matrix.loc['total'] = sorted_matrix.sum()
    sorted_matrix = transpose(sorted_matrix)  ### to transpose CDIV on the row and EMM on the columns

    return(sorted_matrix)


def replace_col_vals(df, tech):
    df = df.copy()
    df.drop(columns=['Technology'], inplace=True)
    df.insert(0, 'Technology', tech)
    return df
    
def convert_matrix(sector,filedir, filename, weathers, mymap):
    if sector == "commercial":
        county_col = "in.nhgis_county_gisjoin"
        # area_col = "weight"
        area_col = "calc.weighted.sqft"
        sec = "Com"
    else:
        county_col = "in.county"
        area_col = "in.units_represented"
        sec = "Res"
        
    combined_map = combine_keys(mymap[sector])
    mykeys = list(combined_map)
    
    for weath in weathers:
        # weath = 'tmy3'
        df_all = pd.read_parquet(f"{filedir}{weath}/{filename}", engine='pyarrow')
        df_all.rename(columns={county_col: 'county'}, inplace=True)
        df_all.rename(columns={'in.state': 'state'}, inplace=True)
        df_all.rename(columns={area_col: "warea"}, inplace=True)
        
        df_all.reset_index(inplace=True)
        df_all = apply_geographies(df_all, scoutgeo_df, geos)
        df_all = df_all.dropna(subset=geos)

        all_eu = pd.DataFrame()
        for eu in mykeys:
            df = df_all.copy()
            df[eu] = df[combined_map[eu]].sum(axis=1)
            df = df[df[eu]>0]
            mapfile = f"map_{sector[:3]}_{eu}.csv"
            map_df = pd.read_csv(f"./{fdir}/mapping/{mapfile}")

            df['scout_tech'] = None
            for _, map_row in map_df.iterrows():
                condition = pd.Series(True, index=df.index)
                for col in map_df.columns:
                    if col != 'scout_tech' and col in df.columns:
                        condition &= (df[col] == map_row[col])
                df.loc[condition, 'scout_tech'] = map_row['scout_tech']
            matched_rows = df[df['scout_tech'].notna()]
            columns_to_keep = geos + ['scout_tech'] + [eu, 'warea']
            df = df[columns_to_keep]
            tech_list = df['scout_tech'].unique().tolist()            
            # df = df.groupby(['emm', 'cdiv', 'scout_tech']).sum().reset_index()
            
            all_tech = pd.DataFrame()
            for tech in tech_list:

                convert_pd = pd.DataFrame()
                tdf = df[(df['scout_tech'] == tech)]
                tdf = tdf.drop(columns=['scout_tech'])
                
                tdf_temp = tdf[['emm', 'cdiv', 'warea', eu]]
                mask = (tdf_temp[[eu]] != 0).all(axis=1)
                tedf = tdf_temp[mask]

                conversion_matrix = tedf.pivot_table(index='emm', columns='cdiv', values="warea", aggfunc='sum')
                convert_matrix = output(conversion_matrix.reset_index())
                convert_pd = convert_matrix if convert_pd.empty else pd.concat([convert_pd, convert_matrix], ignore_index=False)
                normalized_matrix = conversion_matrix.div(conversion_matrix.sum(axis=0), axis=1).reset_index()
                normalized_matrix = output(normalized_matrix)
                normalized_matrix = normalized_matrix.fillna(0)
                normalized_matrix.columns = normalized_matrix.iloc[0]
                normalized_matrix.rename(columns={normalized_matrix.columns[-1]: 'Total'}, inplace=True)
                normalized_matrix = normalized_matrix.iloc[1:]
                normalized_matrix.insert(0, 'CDIV', normalized_matrix.index)
                normalized_matrix.insert(0, 'End use',  eu.split('_')[1])
                normalized_matrix.insert(0, 'Technology', tech)
                if (normalized_matrix['Total'] != 0).all():
                    normalized_matrix.drop(columns=['Total'], inplace=True)
                    all_tech = normalized_matrix if all_tech.empty else pd.concat([all_tech, normalized_matrix], ignore_index=False)
                    if tech == "res_type_central_AC":
                        norm2 = replace_col_vals(normalized_matrix, "wall-window_room_AC")
                        all_tech = pd.concat([all_tech, norm2], ignore_index=False)
            all_eu = all_tech if all_eu.empty else pd.concat([all_eu, all_tech], ignore_index=False)
        all_eu.to_csv(f"./{outdir}/{sec}_Cdiv_EMM_{weath}_electricity_Stock_Tech.csv", index=False)


scoutgeo_df = get_scout_geo()

fdir = 'input'
outdir = 'output/2024_technology'
geos = ['emm', 'cdiv']

print("COMMERCIAL")
convert_matrix(
    sector = "commercial",
    filedir = f"./{fdir}/2024_comstock/",
    filename = "baseline_metadata_and_annual_results.parquet",
    weathers = ['amy2018'],
    mymap = fuel_enduse_map)
print("RESIDENTIAL")
convert_matrix(
    sector = "residential",
    filedir = f"./{fdir}/2024_resstock/",
    filename = "baseline_metadata_and_annual_results.parquet",
    weathers = ['amy2018'],
    mymap = fuel_enduse_map)

print("COMPLETE")


### CDIV/STATE - ENERGY (TECH)

In [ ]:
def apply_geographies(df, dfdict, geos):
    df['county'] = df['county'].astype(str)
    df.loc[df['state'] == 'AK', 'county'] = 'G0'
    df.loc[df['state'] == 'HI', 'county'] = 'G1'        
    for geo in geos:
        geocol = 'cdiv' if geo == 'cdiv' else geo
        d = dfdict.set_index('gisjoin').T.to_dict('index')[geocol]
        df[geo] = df['county'].map(d)
    df = df.drop('county', axis=1)
    return df

def output(df):
    sorted_matrix = df
    sorted_matrix.loc['total'] = sorted_matrix.sum()
    sorted_matrix = transpose(sorted_matrix)  ### to transpose CDIV on the row and EMM on the columns
    return(sorted_matrix)
    
def replace_col_vals(df, tech):
    df = df.copy()
    df.drop(columns=['Technology'], inplace=True)
    df.insert(0, 'Technology', tech)
    return df
    
def convert_matrix(sector,filedir, filename, weathers, mymap):
    if sector == "commercial":
        county_col = "in.nhgis_county_gisjoin"
        sec = "Com"
    else:
        county_col = "in.county"
        sec = "Res"        
    combined_map = combine_keys(mymap[sector])
    mykeys = list(combined_map)
    
    for weath in weathers:
        # weath = 'tmy3'
        df_all = pd.read_parquet(f"{filedir}{weath}/{filename}", engine='pyarrow')
        df_all.rename(columns={county_col: 'county'}, inplace=True)
        df_all.rename(columns={'in.state': 'state'}, inplace=True)
        
        df_all.reset_index(inplace=True)
        df_all = apply_geographies(df_all, scoutgeo_df, geos)
        df_all = df_all.dropna(subset=geos)

        all_eu = pd.DataFrame()
        for eu in mykeys:
            df = df_all.copy()
            df[eu] = df[combined_map[eu]].sum(axis=1)
            df = df[df[eu]>0]
            mapfile = f"map_{sector[:3]}_{eu}.csv"
            map_df = pd.read_csv(f"./{fdir}/mapping/{mapfile}")

            df['scout_tech'] = None
            for _, map_row in map_df.iterrows():
                condition = pd.Series(True, index=df.index)
                for col in map_df.columns:
                    if col != 'scout_tech' and col in df.columns:
                        condition &= (df[col] == map_row[col])
                df.loc[condition, 'scout_tech'] = map_row['scout_tech']
            matched_rows = df[df['scout_tech'].notna()]
            columns_to_keep = geos + ['scout_tech'] + [eu]
            df = df[columns_to_keep]
            tech_list = df['scout_tech'].unique().tolist()            
            df = df.groupby(['state', 'cdiv', 'scout_tech']).sum().reset_index()

            all_tech = pd.DataFrame()
            for tech in tech_list:

                convert_pd = pd.DataFrame()
                tdf = df[(df['scout_tech'] == tech)]
                tdf = tdf.drop(columns=['scout_tech'])
                conversion_matrix = tdf.pivot(index='state', columns='cdiv', values=eu)
                convert_matrix = output(conversion_matrix.reset_index())
                convert_pd = convert_matrix if convert_pd.empty else pd.concat([convert_pd, convert_matrix], ignore_index=False)
                normalized_matrix = conversion_matrix.div(conversion_matrix.sum(axis=0), axis=1).reset_index()
                normalized_matrix = output(normalized_matrix)
                normalized_matrix = normalized_matrix.fillna(0)
                normalized_matrix.columns = normalized_matrix.iloc[0]
                normalized_matrix.rename(columns={normalized_matrix.columns[-1]: 'Total'}, inplace=True)
                normalized_matrix = normalized_matrix.iloc[1:]
                normalized_matrix.insert(0, 'CDIV', normalized_matrix.index)
                normalized_matrix.insert(0, 'End use',  eu.split('_')[1])
                normalized_matrix.insert(0, 'Technology', tech)
                if (normalized_matrix['Total'] != 0).all():
                    normalized_matrix.drop(columns=['Total'], inplace=True)
                    all_tech = normalized_matrix if all_tech.empty else pd.concat([all_tech, normalized_matrix], ignore_index=False)
                    if tech == "res_type_central_AC":
                        norm2 = replace_col_vals(normalized_matrix, "wall-window_room_AC")
                        all_tech = pd.concat([all_tech, norm2], ignore_index=False)   
            all_eu = all_tech if all_eu.empty else pd.concat([all_eu, all_tech], ignore_index=False)
        all_eu.to_csv(f"./{outdir}/{sec}_Cdiv_State_{weath}_electricity_Tech.csv", index=False)


scoutgeo_df = get_scout_geo()

fdir = 'input'
outdir = 'output/2024_technology'
geos = ['cdiv','state']

print("COMMERCIAL")
convert_matrix(
    sector = "commercial",
    filedir = f"./{fdir}/2024_comstock/",
    filename = "baseline_metadata_and_annual_results.parquet",
    weathers = ['amy2018'],
    mymap = fuel_enduse_map)

print("RESIDENTIAL")
convert_matrix(
    sector = "residential",
    filedir = f"./{fdir}/2024_resstock/",
    filename = "baseline_metadata_and_annual_results.parquet",
    weathers = ['amy2018'],
    mymap = fuel_enduse_map)

print("COMPLETE")


### CDIV/STATE - STOCK (TECH)

In [ ]:
def apply_geographies(df, dfdict, geos):
    df['county'] = df['county'].astype(str)
    df.loc[df['state'] == 'AK', 'county'] = 'G0'
    df.loc[df['state'] == 'HI', 'county'] = 'G1'        
    for geo in geos:
        geocol = 'cdiv' if geo == 'cdiv' else geo
        d = dfdict.set_index('gisjoin').T.to_dict('index')[geocol]
        df[geo] = df['county'].map(d)          
    df = df.drop('county', axis=1)
    return df

def output(df):
    sorted_matrix = df
    sorted_matrix.loc['total'] = sorted_matrix.sum()
    sorted_matrix = transpose(sorted_matrix)  ### to transpose CDIV on the row and EMM on the columns
    return(sorted_matrix)


def replace_col_vals(df, tech):
    df = df.copy()
    df.drop(columns=['Technology'], inplace=True)
    df.insert(0, 'Technology', tech)
    return df
    
def convert_matrix(sector,filedir, filename, weathers, mymap):        
    if sector == "commercial":
        county_col = "in.nhgis_county_gisjoin"
        # area_col = "weight"
        area_col = "calc.weighted.sqft"
        sec = "Com"
    else:
        county_col = "in.county"
        area_col = "in.units_represented"
        sec = "Res"
    
    combined_map = combine_keys(mymap[sector])
    mykeys = list(combined_map)
    
    for weath in weathers:
        # weath = 'tmy3'
        df_all = pd.read_parquet(f"{filedir}{weath}/{filename}", engine='pyarrow')
        df_all.rename(columns={county_col: 'county'}, inplace=True)
        df_all.rename(columns={'in.state': 'state'}, inplace=True)
        df_all.rename(columns={area_col: "warea"}, inplace=True)
        
        df_all.reset_index(inplace=True)
        df_all = apply_geographies(df_all, scoutgeo_df, geos)
        df_all = df_all.dropna(subset=geos)

        all_eu = pd.DataFrame()
        for eu in mykeys:
            df = df_all.copy()
            # print(f"END USE= {eu}")
            df[eu] = df[combined_map[eu]].sum(axis=1)
            df = df[df[eu]>0]
            mapfile = f"map_{sector[:3]}_{eu}.csv"
            map_df = pd.read_csv(f"./{fdir}/mapping/{mapfile}")

            df['scout_tech'] = None
            for _, map_row in map_df.iterrows():
                condition = pd.Series(True, index=df.index)
                for col in map_df.columns:
                    if col != 'scout_tech' and col in df.columns:
                        condition &= (df[col] == map_row[col])
                df.loc[condition, 'scout_tech'] = map_row['scout_tech']
            matched_rows = df[df['scout_tech'].notna()]
            columns_to_keep = geos + ['scout_tech'] + [eu,'warea']
            df = df[columns_to_keep]
            tech_list = df['scout_tech'].unique().tolist()
            # df = df.groupby(['state', 'cdiv', 'scout_tech']).sum().reset_index()
            all_tech = pd.DataFrame()
            for tech in tech_list:

                convert_pd = pd.DataFrame()
                tdf = df[(df['scout_tech'] == tech)]
                tdf = tdf.drop(columns=['scout_tech'])
                
                tdf_temp = tdf[['state', 'cdiv', 'warea', eu]]
                mask = (tdf_temp[[eu]] != 0).all(axis=1)
                tedf = tdf_temp[mask]

                conversion_matrix = tedf.pivot_table(index='state', columns='cdiv', values="warea", aggfunc='sum')
                convert_matrix = output(conversion_matrix.reset_index())
                convert_pd = convert_matrix if convert_pd.empty else pd.concat([convert_pd, convert_matrix], ignore_index=False)
                normalized_matrix = conversion_matrix.div(conversion_matrix.sum(axis=0), axis=1).reset_index()
                normalized_matrix = output(normalized_matrix)
                normalized_matrix = normalized_matrix.fillna(0)
                normalized_matrix.columns = normalized_matrix.iloc[0]
                normalized_matrix.rename(columns={normalized_matrix.columns[-1]: 'Total'}, inplace=True)
                normalized_matrix = normalized_matrix.iloc[1:]
                normalized_matrix.insert(0, 'CDIV', normalized_matrix.index)
                normalized_matrix.insert(0, 'End use',  eu.split('_')[1])
                normalized_matrix.insert(0, 'Technology', tech)
                if (normalized_matrix['Total'] != 0).all():
                    normalized_matrix.drop(columns=['Total'], inplace=True)
                    all_tech = normalized_matrix if all_tech.empty else pd.concat([all_tech, normalized_matrix], ignore_index=False)
                    if tech == "res_type_central_AC":
                        norm2 = replace_col_vals(normalized_matrix, "wall-window_room_AC")
                        all_tech = pd.concat([all_tech, norm2], ignore_index=False)
            all_eu = all_tech if all_eu.empty else pd.concat([all_eu, all_tech], ignore_index=False)
        all_eu.to_csv(f"./{outdir}/{sec}_Cdiv_State_{weath}_electricity_Stock_Tech.csv", index=False)


scoutgeo_df = get_scout_geo()

fdir = 'input'
outdir = 'output/2024_technology'
geos = ['cdiv','state']

print("COMMERCIAL")
convert_matrix(
    sector = "commercial",
    filedir = f"./{fdir}/2024_comstock/",
    filename = "baseline_metadata_and_annual_results.parquet",
    weathers = ['amy2018'],
    mymap = fuel_enduse_map)
print("RESIDENTIAL")
convert_matrix(
    sector = "residential",
    filedir = f"./{fdir}/2024_resstock/",
    filename = "baseline_metadata_and_annual_results.parquet",
    weathers = ['amy2018'],
    mymap = fuel_enduse_map)

print("COMPLETE")


### CDIV/EMM - ENERGY (END USE)

In [ ]:
#### Energy ####
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

def apply_geographies(df, dfdict, geos):
    df['county'] = df['county'].astype(str)
    # df.loc[df['state'] == 'AK', 'county'] = 'G0'
    # df.loc[df['state'] == 'HI', 'county'] = 'G1'        
    for geo in geos:
        geocol = 'emm2020_county' if geo == 'emm' else geo
        d = dfdict.set_index('gisjoin').T.to_dict('index')[geocol]
        df[geo] = df['county'].map(d)
    df = df.drop('county', axis=1)
    return df

def output(df):
    merged_matrix = df.merge(df_order, left_on='emm', right_on='emm_intersect', how='left')
    sorted_matrix = merged_matrix.sort_values(by='no')
    sorted_matrix = sorted_matrix.drop(sorted_matrix.columns[-2:], axis=1)
    sorted_matrix.loc['total'] = sorted_matrix.sum()
    sorted_matrix = transpose(sorted_matrix)  ### to transpose CDIV on the row and EMM on the columns

    return(sorted_matrix)


def convert_matrix(sector,filedir, filename, weathers, mymap, fueltype):
    if sector == "commercial":
        county_col = "in.nhgis_county_gisjoin"
        sec = "Com"
    else:
        county_col = "in.county"
        sec = "Res"
    
    mykeys = list(mymap[sector][fueltype])

    for weath in weathers:
        df = pd.read_parquet(f"{filedir}{weath}/{filename}", engine='pyarrow')
        df.rename(columns={county_col: 'county'}, inplace=True)
        df.rename(columns={'in.state': 'state'}, inplace=True)

        for eu in mykeys:
            df[eu] = df[mymap[sector][fueltype][eu]].sum(axis=1)
        df.reset_index(inplace=True)
        df = apply_geographies(df, scoutgeo_df, geos)
        df = df.dropna(subset=geos)
        df = df[geos+mykeys]
        df = df.groupby(['emm','cdiv']).sum().reset_index()
        norm_pd = pd.DataFrame()
        convert_pd = pd.DataFrame()
        for eu in mykeys:

            conversion_matrix = df.pivot(index='emm', columns='cdiv', values=eu)
            convert_matrix = output(conversion_matrix.reset_index())
            convert_pd = convert_matrix if convert_pd.empty else pd.concat([convert_pd, convert_matrix], ignore_index=False)
            normalized_matrix = conversion_matrix.div(conversion_matrix.sum(axis=0), axis=1).reset_index()
            normalized_matrix = output(normalized_matrix)
            normalized_matrix = normalized_matrix.fillna(0)
            normalized_matrix.columns = normalized_matrix.iloc[0]
            normalized_matrix.rename(columns={normalized_matrix.columns[-1]: 'Total'}, inplace=True)
            normalized_matrix = normalized_matrix.iloc[1:]
            normalized_matrix.insert(0, 'CDIV', normalized_matrix.index)
            normalized_matrix.insert(0, 'End use', eu)
            normalized_matrix.drop(columns=['Total'], inplace=True)
            norm_pd = normalized_matrix if norm_pd.empty else pd.concat([norm_pd, normalized_matrix], ignore_index=False)
        if sec == "Res": norm_pd['AK_HI'] = 0    
        norm_pd.to_csv(f"./{outdir}/{sec}_Cdiv_EMM_{weath}_{remove_space(fueltype)}.csv", index=False)

scoutgeo_df = get_scout_geo()

fdir = 'input'
outdir = 'output/2024_end_use'
building_types = ['commercial','residential']
geos = ['emm', 'cdiv']
fuel_com = ['electricity', 'natural gas', 'distillate', 'other fuel']
fuel_res = ['electricity', 'natural gas', 'distillate', 'other fuel']
################################################################################
print("COMMERCIAL")

for fl in fuel_com:
    convert_matrix(
        sector = "commercial",
        filedir = f"./{fdir}/2024_comstock/",
        filename = "baseline_metadata_and_annual_results.parquet",
        weathers = ['amy2018'],
        mymap = enduse_map,
        fueltype = fl)

print("RESIDENTIAL")
for fl in fuel_res:
    convert_matrix(
        sector = "residential",
        filedir = f"./{fdir}/2024_resstock/",
        filename = "baseline_metadata_and_annual_results.parquet",
        weathers = ['amy2018'],#,'tmy3'],
        mymap = enduse_map,
        fueltype = fl)

print("COMPLETE")

### CDIV/EMM - STOCK (END USE)

In [ ]:
def apply_geographies(df, dfdict, geos):
    df['county'] = df['county'].astype(str)
    # df.loc[df['state'] == 'AK', 'county'] = 'G0'
    # df.loc[df['state'] == 'HI', 'county'] = 'G1'        
    for geo in geos:
        geocol = 'emm2020_county' if geo == 'emm' else geo
        d = dfdict.set_index('gisjoin').T.to_dict('index')[geocol]
        df[geo] = df['county'].map(d)
    df = df.drop('county', axis=1)
    return df

def output(df):
    merged_matrix = df.merge(df_order, left_on='emm', right_on='emm_intersect', how='left')
    sorted_matrix = merged_matrix.sort_values(by='no')
    sorted_matrix = sorted_matrix.drop(sorted_matrix.columns[-2:], axis=1)
    sorted_matrix.loc['total'] = sorted_matrix.sum()
    sorted_matrix = transpose(sorted_matrix)  ### to transpose CDIV on the row and EMM on the columns

    return(sorted_matrix)
    
def eu_rows(df, category, columns_dict, threshold=1):
    # threshold means, minimum 1 col in the dict is non-zero
    columns = columns_dict.get(category, [])
    mask = (df[columns] != 0).sum(axis=1) >= threshold
    filtered_df = df[mask]
    return filtered_df

def eu_rows_woutthreshold(df, category, columns_dict):
    columns = columns_dict.get(category, [])
    mask = (df[columns] != 0).all(axis=1)
    filtered_df = df[mask]
    return filtered_df
    
def convert_matrix_stock(sector,filedir, filename, weathers, mymap, fueltype):
    conditions_dict = mymap[sector][fueltype]
        
    if sector == "commercial":
        sec = "Com"
        county_col = "in.nhgis_county_gisjoin"
        # area_col = "weight"
        area_col = "calc.weighted.sqft"
    else:
        sec = "Res"
        county_col = "in.county"
        area_col = "in.units_represented"
        # area_col = "weight"
    
    for weath in weathers:    
        alldf = pd.read_parquet(f"{filedir}{weath}/{filename}", engine='pyarrow')
        alldf.rename(columns={county_col: "county"}, inplace=True)
        alldf.rename(columns={"in.state": "state"}, inplace=True)
        alldf.rename(columns={area_col: "warea"}, inplace=True)
        
        mykeys = list(mymap[sector][fueltype])
        
        norm_pd = pd.DataFrame()
        convert_pd = pd.DataFrame()
        
        for eu in mykeys:
            df = eu_rows(alldf, eu, conditions_dict)
    
            df = apply_geographies(df, scoutgeo_df, geos)
            df = df[["warea","emm","cdiv"]] #"state","weight","in.sqft"]]
            
            conversion_matrix = df.pivot_table(index='emm', columns='cdiv', values="warea", aggfunc='sum')
            convert_matrix = output(conversion_matrix.reset_index())
            convert_pd = convert_matrix if convert_pd.empty else pd.concat([convert_pd, convert_matrix], ignore_index=False)
            normalized_matrix = conversion_matrix.div(conversion_matrix.sum(axis=0), axis=1).reset_index()
            normalized_matrix = output(normalized_matrix)
            normalized_matrix = normalized_matrix.fillna(0)
            normalized_matrix.columns = normalized_matrix.iloc[0]
            normalized_matrix.rename(columns={normalized_matrix.columns[-1]: 'Total'}, inplace=True)
            normalized_matrix = normalized_matrix.iloc[1:]
            normalized_matrix.insert(0, 'CDIV', normalized_matrix.index)
            normalized_matrix.insert(0, 'End use', eu)
            normalized_matrix.drop(columns=['Total'], inplace=True)
            norm_pd = normalized_matrix if norm_pd.empty else pd.concat([norm_pd, normalized_matrix], ignore_index=False)
        if sec == "Res": norm_pd['AK_HI'] = 0     
        norm_pd.to_csv(f"./{outdir}/{sec}_Cdiv_EMM_{weath}_{remove_space(fueltype)}_Stock.csv", index=False)
################################################################################
scoutgeo_df = get_scout_geo()

fdir = 'input'
outdir = 'output/2024_end_use'
building_types = ['commercial','residential']
geos = ['emm', 'cdiv']
fuel_com = ['electricity', 'natural gas', 'distillate', 'other fuel']
fuel_res = ['electricity', 'natural gas', 'distillate', 'other fuel']

print("COMMERCIAL")
for fl in fuel_com:
    convert_matrix_stock(
        sector = "commercial",
        filedir = f"./{fdir}/2024_comstock/",
        filename = "baseline_metadata_and_annual_results.parquet",
        weathers = ['amy2018'],
        mymap = enduse_map,
        fueltype =fl
    )

print("RESIDENTIAL")
for fl in fuel_res:
    convert_matrix_stock(
        sector = "residential",
        filedir = f"./{fdir}/2024_resstock/",
        filename = "baseline_metadata_and_annual_results.parquet",
        weathers = ['amy2018'],
        mymap = enduse_map,
        fueltype = fl
    )

print("COMPLETE")

### CDIV/STATE - ENERGY (END USE)

In [ ]:
### Energy ###
### ResStock 2022 ComStock 2023
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

def apply_geographies(df, dfdict, geos):
    df['county'] = df['county'].astype(str)
    # df.loc[df['state'] == 'AK', 'county'] = 'G0'
    # df.loc[df['state'] == 'HI', 'county'] = 'G1'
    for geo in geos:
        geocol = 'cdiv' if geo == 'cdiv' else geo
        d = dfdict.set_index('gisjoin').T.to_dict('index')[geocol]
        df[geo] = df['county'].map(d)          
    df = df.drop('county', axis=1)
    # print((df[df['state'].isnull()])['state'])
    return df

def output(df):
    # merged_matrix = df.merge(df_order, left_on='cdiv', right_on='cdiv_intersect', how='left')
    # sorted_matrix = merged_matrix.sort_values(by='no')
    # sorted_matrix = sorted_matrix.drop(sorted_matrix.columns[-2:], axis=1)
    sorted_matrix = df
    sorted_matrix.loc['total'] = sorted_matrix.sum()
    sorted_matrix = transpose(sorted_matrix)  ### to transpose CDIV on the row and EMM on the columns
    return(sorted_matrix)


def convert_matrix(sector,filedir, filename, weathers, mymap, fueltype):
    if sector == "commercial":
        county_col = "in.nhgis_county_gisjoin"
        sec = "Com"
    else:
        county_col = "in.county"
        sec = "Res"
    
    mykeys = list(mymap[sector][fueltype])
    for weath in weathers:
        df = pd.read_parquet(f"{filedir}{weath}/{filename}", engine='pyarrow')

        # list_ak_counties = (df[df['in.state'] == 'AK']['in.county']).to_list()
        # list_hi_counties = (df[df['in.state'] == 'hi']['in.county']).to_list()
        # filtered_values = df[df['in.county'].isin(list_hi_counties)]['in.county']
        # for value in filtered_values:
        #     print(value)
        
        df.rename(columns={county_col: 'county'}, inplace=True)
        df.rename(columns={'in.state': 'state'}, inplace=True)
        for eu in mykeys:
            df[eu] = df[mymap[sector][fueltype][eu]].sum(axis=1)
        df.reset_index(inplace=True)
        df = apply_geographies(df, scoutgeo_df, geos)
        df = df.dropna(subset=geos)
        df = df[geos+mykeys]
        df = df.groupby(['cdiv', 'state']).sum().reset_index()
        norm_pd = pd.DataFrame()
        convert_pd = pd.DataFrame()
        for eu in mykeys:
            conversion_matrix = df.pivot(index='state', columns='cdiv', values=eu)
            convert_matrix = output(conversion_matrix.reset_index())
            convert_pd = convert_matrix if convert_pd.empty else pd.concat([convert_pd, convert_matrix], ignore_index=False)
            normalized_matrix = conversion_matrix.div(conversion_matrix.sum(axis=0), axis=1).reset_index()
            normalized_matrix = output(normalized_matrix)
            
            normalized_matrix = normalized_matrix.fillna(0)
            normalized_matrix.columns = normalized_matrix.iloc[0]
            normalized_matrix.rename(columns={normalized_matrix.columns[-1]: 'Total'}, inplace=True)
            normalized_matrix = normalized_matrix.iloc[1:]
            normalized_matrix.insert(0, 'CDIV', normalized_matrix.index)
            normalized_matrix.insert(0, 'End use', eu)
            normalized_matrix.drop(columns=['Total'], inplace=True)
            norm_pd = normalized_matrix if norm_pd.empty else pd.concat([norm_pd, normalized_matrix], ignore_index=False)
        if sec == "Res":
            norm_pd['AK'] = 0
            norm_pd['HI'] = 0
        norm_pd.to_csv(f"./{outdir}/{sec}_Cdiv_State_{weath}_{remove_space(fueltype)}.csv", index=False)

scoutgeo_df = get_scout_geo()

fdir = 'input'
outdir = 'output/2024_end_use'
building_types = ['commercial','residential']
lst_ak_counties = []
lst_hi_counties = []

geos = ['cdiv', 'state']
fuel_com = ['electricity', 'natural gas', 'distillate', 'other fuel']
fuel_res = ['electricity', 'natural gas', 'distillate', 'other fuel']
################################################################################
print("COMMERCIAL")
for fl in fuel_com:
    convert_matrix(
        sector = "commercial",
        filedir = f"./{fdir}/2024_comstock/",
        filename = "baseline_metadata_and_annual_results.parquet",
        weathers = ['amy2018'],
        mymap = enduse_map,
        fueltype = fl
    )
print("RESIDENTIAL")
for fl in fuel_res:
    convert_matrix(
        sector = "residential",
        filedir = f"./{fdir}/2024_resstock/",
        filename = "baseline_metadata_and_annual_results.parquet",
        weathers = ['amy2018'],
        mymap = enduse_map,
        fueltype = fl
    )
################################################################################
print ("COMPLETE!")

### CDIV/STATE - STOCK (END USE)

In [ ]:
def apply_geographies(df, dfdict, geos):
    df['county'] = df['county'].astype(str)
    # df.loc[df['state'] == 'AK', 'county'] = 'G0'
    # df.loc[df['state'] == 'HI', 'county'] = 'G1'
    for geo in geos:
        geocol = 'cdiv' if geo == 'cdiv' else geo
        d = dfdict.set_index('gisjoin').T.to_dict('index')[geocol]
        df[geo] = df['county'].map(d)          
    df = df.drop('county', axis=1)
    # print((df[df['state'].isnull()])['state'])
    return df

def output(df):
    # merged_matrix = df.merge(df_order, left_on='cdiv', right_on='cdiv_intersect', how='left')
    # sorted_matrix = merged_matrix.sort_values(by='no')
    # sorted_matrix = sorted_matrix.drop(sorted_matrix.columns[-2:], axis=1)
    sorted_matrix = df
    sorted_matrix.loc['total'] = sorted_matrix.sum()
    sorted_matrix = transpose(sorted_matrix)  ### to transpose CDIV on the row and EMM on the columns
    return(sorted_matrix)


def eu_rows(df, category, columns_dict, threshold=1):
    # threshold means, minimum 1 col in the dict is non-zero
    columns = columns_dict.get(category, [])
    mask = (df[columns] != 0).sum(axis=1) >= threshold
    filtered_df = df[mask]
    return filtered_df

def eu_rows_woutthreshold(df, category, columns_dict):
    columns = columns_dict.get(category, [])
    mask = (df[columns] != 0).all(axis=1)
    filtered_df = df[mask]
    return filtered_df
    
def convert_matrix_stock(sector,filedir, filename, weathers, mymap, fueltype):
    conditions_dict = mymap[sector][fueltype]
    if sector == "commercial":
        sec = "Com"
        county_col = "in.nhgis_county_gisjoin"
        # area_col = "weight"
        area_col = "calc.weighted.sqft"
    else:
        sec = "Res"
        county_col = "in.county"
        area_col = "in.units_represented"
        # area_col = "weight"
        
    for weath in weathers:    
        alldf = pd.read_parquet(f"{filedir}{weath}/{filename}", engine='pyarrow')
        alldf.rename(columns={county_col: "county"}, inplace=True)
        alldf.rename(columns={"in.state": "state"}, inplace=True)
        alldf.rename(columns={area_col: "warea"}, inplace=True)
        
        mykeys = list(mymap[sector][fueltype])
        
        norm_pd = pd.DataFrame()
        convert_pd = pd.DataFrame()
        
        for eu in mykeys:
            df = eu_rows(alldf, eu, conditions_dict)
            df = apply_geographies(df, scoutgeo_df, geos)
            # df = df.dropna(subset=geos)
            df = df[["warea","state","cdiv"]] #"state","weight","in.sqft"]]
            conversion_matrix = df.pivot_table(index='state', columns='cdiv', values="warea", aggfunc='sum')
            convert_matrix = output(conversion_matrix.reset_index())
            convert_pd = convert_matrix if convert_pd.empty else pd.concat([convert_pd, convert_matrix], ignore_index=False)
            normalized_matrix = conversion_matrix.div(conversion_matrix.sum(axis=0), axis=1).reset_index()
            normalized_matrix = output(normalized_matrix)
            normalized_matrix = normalized_matrix.fillna(0)
            normalized_matrix.columns = normalized_matrix.iloc[0]
            normalized_matrix.rename(columns={normalized_matrix.columns[-1]: 'Total'}, inplace=True)
            normalized_matrix = normalized_matrix.iloc[1:]
            normalized_matrix.insert(0, 'CDIV', normalized_matrix.index)
            normalized_matrix.insert(0, 'End use', eu)
            normalized_matrix.drop(columns=['Total'], inplace=True)
            norm_pd = normalized_matrix if norm_pd.empty else pd.concat([norm_pd, normalized_matrix], ignore_index=False)
        if sec == "Res": 
            norm_pd['AK'] = 0
            norm_pd['HI'] = 0    
        norm_pd.to_csv(f"./{outdir}/{sec}_Cdiv_State_{weath}_{remove_space(fueltype)}_Stock.csv", index=False)
################################################################################
scoutgeo_df = get_scout_geo()

fdir = 'input'
outdir = 'output/2024_end_use'
building_types = ['commercial','residential']
lst_ak_counties = []
lst_hi_counties = []

geos = ['cdiv', 'state']
fuel_com = ['electricity', 'natural gas', 'distillate', 'other fuel']
fuel_res = ['electricity', 'natural gas', 'distillate', 'other fuel']
################################################################################'
building_types = ['commercial','residential']
lst_ak_counties = []
lst_hi_counties = []

geos = ['cdiv', 'state']

print("COMMERCIAL")
for fl in fuel_com:
    convert_matrix_stock(
        sector = "commercial",
        filedir = f"./{fdir}/2024_comstock/",
        filename = "baseline_metadata_and_annual_results.parquet",
        weathers = ['amy2018'],
        mymap = enduse_map,
        fueltype = fl
    )
print("RESIDENTIAL")
for fl in fuel_res:
    convert_matrix_stock(
        sector = "residential",
        filedir = f"./{fdir}/2024_resstock/",
        filename = "baseline_metadata_and_annual_results.parquet",
        weathers = ['amy2018'],
        mymap = enduse_map,
        fueltype = fl
    )

print("COMPLETE")

### STATE/EMM - ENERGY (END USE)

In [ ]:
### ResStock 2024 ComStock 2024
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

def apply_geographies(df, dfdict, geos):
    df['county'] = df['county'].astype(str)
    df.loc[df['state'] == 'AK', 'county'] = 'G0'
    df.loc[df['state'] == 'HI', 'county'] = 'G1'    
    for geo in geos:
        geocol = 'emm2020_county' if geo == 'emm' else geo
        d = dfdict.set_index('gisjoin').T.to_dict('index')[geocol]
        df[geo] = df['county'].map(d)
    df = df.drop('county', axis=1)
    return df

def output(df):
    merged_matrix = df.merge(df_order, left_on='emm', right_on='emm_intersect', how='left')
    sorted_matrix = merged_matrix.sort_values(by='no')
    sorted_matrix = sorted_matrix.drop(sorted_matrix.columns[-2:], axis=1)
    sorted_matrix.loc['total'] = sorted_matrix.sum()
    sorted_matrix = transpose(sorted_matrix)
    return(sorted_matrix)


def convert_matrix(sector,filedir, filename, weathers, mymap):
    if sector == "commercial":
        county_col = "in.nhgis_county_gisjoin"
        sec = "Com"
    else:
        county_col = "in.county"
        sec = "Res"
    
    mykeys = list(mymap[sector])
    for weath in weathers:
        df = pd.read_parquet(f"{filedir}{weath}/{filename}", engine='pyarrow')

        # list_ak_counties = (df[df['in.state'] == 'AK']['in.nhgis_county_gisjoin']).to_list()
        # list_hi_counties = (df[df['in.state'] == 'hi']['in.nhgis_county_gisjoin']).to_list()
        # filtered_values = df[df['in.county'].isin(list_hi_counties)]['in.county']
        # for value in filtered_values:
        #     print(value)
        
        df.rename(columns={county_col: 'county'}, inplace=True)
        df.rename(columns={'in.state': 'state'}, inplace=True)
        for eu in mykeys:
            df[eu] = df[mymap[sector][eu]].sum(axis=1)
        df.reset_index(inplace=True)
        df = apply_geographies(df, scoutgeo_df, geos)
        df = df.dropna(subset=geos)
        df = df[geos+mykeys]
        df = df.groupby(['emm', 'state']).sum().reset_index()
        norm_pd = pd.DataFrame()
        convert_pd = pd.DataFrame()
        for eu in mykeys:
            # sdf = df[['emm','cdiv','total_elec']]
            conversion_matrix = df.pivot(index='emm', columns='state', values=eu)
            convert_matrix = output(conversion_matrix.reset_index())
            convert_pd = convert_matrix if convert_pd.empty else pd.concat([convert_pd, convert_matrix], ignore_index=False)
            normalized_matrix = conversion_matrix.div(conversion_matrix.sum(axis=0), axis=1).reset_index()
            normalized_matrix = output(normalized_matrix)

            normalized_matrix = normalized_matrix.fillna(0)
            normalized_matrix.columns = normalized_matrix.iloc[0]
            normalized_matrix.rename(columns={normalized_matrix.columns[-1]: 'Total'}, inplace=True)
            normalized_matrix = normalized_matrix.iloc[1:]
            normalized_matrix.insert(0, 'State', normalized_matrix.index)
            normalized_matrix.insert(0, 'End use', eu)
            normalized_matrix.drop(columns=['Total'], inplace=True)
            norm_pd = normalized_matrix if norm_pd.empty else pd.concat([norm_pd, normalized_matrix], ignore_index=False)
        norm_pd.to_csv(f"./{outdir}/{sec}_State_EMM_{weath}.csv", index=False)

scoutgeo_df = get_scout_geo()

fdir = 'input'
outdir = 'output/2024_end_use'
building_types = ['commercial','residential']


geos = ['emm', 'state']
################################################################################

print("COMMERCIAL")
convert_matrix(
    sector = "commercial",
    filedir = f"./{fdir}/2024_comstock/",
    filename = "baseline_metadata_and_annual_results.parquet",
    weathers = ['amy2018'],
    mymap = enduse_map)
print("RESIDENTIAL")
convert_matrix(
    sector = "residential",
    filedir = f"./{fdir}/2024_resstock/",
    filename = "baseline_metadata_and_annual_results.parquet",
    weathers = ['tmy3','amy2018'],
    mymap = enduse_map)
################################################################################

print("Complete")

### STATE/EMM - STOCK (END USE)

In [ ]:
def eu_rows(df, category, columns_dict, threshold=1):
    # threshold means, minimum 1 col in the dict is non-zero
    columns = columns_dict.get(category, [])
    mask = (df[columns] != 0).sum(axis=1) >= threshold
    filtered_df = df[mask]
    return filtered_df

def eu_rows_woutthreshold(df, category, columns_dict):
    columns = columns_dict.get(category, [])
    mask = (df[columns] != 0).all(axis=1)
    filtered_df = df[mask]
    return filtered_df
    
def convert_matrix_stock(sector,filedir, filename, weathers, mymap):
    conditions_dict = mymap[sector]
    if sector == "commercial":
        county_col = "in.nhgis_county_gisjoin"
        area_col = "calc.weighted.sqft"
        sec = "Com"
    else:
        county_col = "in.county"
        area_col = "weight"
        sec = "Res"
        
    for weath in weathers:    
        alldf = pd.read_parquet(f"{filedir}{weath}/{filename}", engine='pyarrow')
        alldf.rename(columns={county_col: "county"}, inplace=True)
        alldf.rename(columns={"in.state": "state"}, inplace=True)
        alldf.rename(columns={area_col: "warea"}, inplace=True)
        
        mykeys = list(mymap[sector])
        
        norm_pd = pd.DataFrame()
        convert_pd = pd.DataFrame()
        
        for eu in mykeys:
            df = eu_rows(alldf, eu, conditions_dict)
            df = apply_geographies(df, scoutgeo_df, geos)
            # df = df.dropna(subset=geos)
            df = df[["warea","state","emm"]] #"state","weight","in.sqft"]]
            
            conversion_matrix = df.pivot_table(index='emm', columns='state', values="warea", aggfunc='sum')
            convert_matrix = output(conversion_matrix.reset_index())
            convert_pd = convert_matrix if convert_pd.empty else pd.concat([convert_pd, convert_matrix], ignore_index=False)
            normalized_matrix = conversion_matrix.div(conversion_matrix.sum(axis=0), axis=1).reset_index()
            normalized_matrix = output(normalized_matrix)
            normalized_matrix = normalized_matrix.fillna(0)
            normalized_matrix.columns = normalized_matrix.iloc[0]
            normalized_matrix.rename(columns={normalized_matrix.columns[-1]: 'Total'}, inplace=True)
            normalized_matrix = normalized_matrix.iloc[1:]
            normalized_matrix.insert(0, 'emm', normalized_matrix.index)
            normalized_matrix.insert(0, 'End use', eu)
            norm_pd = normalized_matrix if norm_pd.empty else pd.concat([norm_pd, normalized_matrix], ignore_index=False)
            
        norm_pd.to_csv(f"./{outdir}/{sec}_State_EMM_{weath}_Stock.csv", index=False)
################################################################################
scoutgeo_df = get_scout_geo()

fdir = 'input'
outdir = 'output/2024_end_use'
building_types = ['commercial','residential']


geos = ['emm', 'state']
print("COMMERCIAL")
convert_matrix_stock(
    sector = "commercial",
    filedir = f"./{fdir}/2024_comstock/",
    filename = "baseline_metadata_and_annual_results.parquet",
    weathers = ['amy2018'],
    mymap = enduse_map)
print("RESIDENTIAL")
convert_matrix_stock(
    sector = "residential",
    filedir = f"./{fdir}/2024_resstock/",
    filename = "baseline_metadata_and_annual_results.parquet",
    weathers = ['tmy3','amy2018'],
    mymap = enduse_map)

print("COMPLETE")